# Get Atlas Release Dataset

## Imports

In [ ]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup
Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="bbp/atlas")

## Set the atlas release id
These atlas releases can be explored through the atlas web app:

* dev: https://bluebrainatlas.kcpdev.bbp.epfl.ch/atlas
* prod: https://bbp.epfl.ch/atlas


In [31]:

Prod_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885" 
Staging_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/brainatlasrelease/c96c71a8-4c0d-4bc1-8a1a-141d9ed6693d" 
atlas_release_id = Staging_BBP_Mouse_Brain_Atlas_Release

## Get the atlas release high level metadata

In [ ]:
atlas_release = forge.retrieve(atlas_release_id, version=1)

In [ ]:
print(atlas_release)

In [ ]:
atlas_release._store_metadata["_rev"]

## Get the atlas hierarchy

In [ ]:
parcellation_ontology = forge.retrieve(atlas_release.parcellationOntology.id, cross_bucket=True)

In [ ]:
print(parcellation_ontology)

In [ ]:
r = forge.retrieve(id="https://bbp.epfl.ch/neurosciencegraph/data/ontologies/34388d3b-0b88-4deb-9686-6fcd9ef8990e")
r.distribution = forge.attach("/Users/mfsy/Documents/hierarchy_l23split.json", content_type="application/json")
forge.update(r)

In [21]:
forge.download(parcellation_ontology, "distribution.contentUrl", ".", overwrite=True, cross_bucket=True)

## Get parcellation (annotation) volume

In [ ]:
parcellation_volume = forge.retrieve(atlas_release.parcellationVolume.id)

In [ ]:
print(parcellation_volume)

In [16]:
forge.download(parcellation_volume, "distribution.contentUrl", ".", overwrite=True)

## Get orientation field volume

In [22]:
query = {
          "type":"CellOrientationField", 
          "atlasRelease":{"@id":atlas_release_id},
          "brainLocation":{"brainRegion":{"id":"http://api.brain-map.org/api/v2/data/Structure/997"}} # root brain region
        }
cell_orientation_field = forge.search(query)
print(f"{len(cell_orientation_field)} found")

2 found


In [ ]:
print(cell_orientation_field)

In [23]:
forge.download(cell_orientation_field, "distribution.contentUrl", ".", overwrite=True)

## Get the metype density nrrd file for each region (region is an input)

In [24]:
query = f"""

SELECT ?mtype_label ?nrrd_file ?contentUrl 

WHERE {{
        ?s a MTypeDensity ;
            atlasRelease <{atlas_release_id}>; 
            annotation / hasBody / label ?mtype_label ;
            distribution ?distribution .
        ?distribution name ?nrrd_file ;
            contentUrl ?contentUrl .
}}
"""

In [25]:
resources = forge.sparql(query, limit=1000)

In [26]:
df = forge.as_dataframe(resources)

In [29]:
df.head()

,contentUrl,mtype_label,nrrd_file
0,https://staging.nise.bbp.epfl.ch/nexus/v1/file...,L1_HAC,L1_HAC-BNAC_densities_v3.nrrd
1,https://staging.nise.bbp.epfl.ch/nexus/v1/file...,L23_BP,L23_BP-BIR_densities_v3.nrrd
2,https://staging.nise.bbp.epfl.ch/nexus/v1/file...,L23_DBC,L23_DBC-BNAC_densities_v3.nrrd
3,https://staging.nise.bbp.epfl.ch/nexus/v1/file...,L5_NGC,L5_NGC-BNAC_densities_v3.nrrd
4,https://staging.nise.bbp.epfl.ch/nexus/v1/file...,L5_DBC,L5_DBC-BSTUT_densities_v3.nrrd


In [30]:
forge.download(resources, "contentUrl", ".", overwrite=True)